In [2]:
# Test TFLite model with all images in 'test images' directory
from tensorflow import lite as tflite
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# Class names (must match training order)
class_names = ['BacterialBlights', 'Healthy', 'Mosaic', 'RedRot', 'Rust', 'Yellow']

# Load TFLite model
interpreter = tflite.Interpreter(model_path='sugarcane_model.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Directory with test images
test_dir = 'test images'

# Get list of image files
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
image_files = [f for f in os.listdir(test_dir) if f.lower().endswith(image_extensions)]

# Test each image
for img_file in image_files:
    sample_img_path = os.path.join(test_dir, img_file)

    # Load and preprocess image
    img = image.load_img(sample_img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    # Run inference
    interpreter.set_tensor(input_details[0]['index'], img_array.astype(np.float32))
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])

    # Get prediction
    predicted_index = np.argmax(output)
    predicted_class = class_names[predicted_index]
    confidence = output[0][predicted_index]

    print(f'Image: {img_file}')
    print(f'Predicted Class: {predicted_class}')
    print(f'Confidence: {confidence:.4f}')
    print(f'All Probabilities: {output[0]}')
    print('-' * 50)

Image: cropped_mosaic (11).jpeg
Predicted Class: Mosaic
Confidence: 0.5253
All Probabilities: [1.6630978e-05 3.9411995e-01 5.2526534e-01 1.0953485e-02 6.3403487e-02
 6.2410957e-03]
--------------------------------------------------
Image: moos image.jpg
Predicted Class: RedRot
Confidence: 0.8218
All Probabilities: [1.3120713e-02 2.7136489e-06 3.5421390e-08 8.2182372e-01 1.0042624e-05
 1.6504274e-01]
--------------------------------------------------
Image: ru.jpg
Predicted Class: Rust
Confidence: 0.9965
All Probabilities: [3.3768662e-03 5.9475505e-09 7.0545948e-06 9.6525022e-05 9.9651778e-01
 1.8852833e-06]
--------------------------------------------------
